# 5. Cell Type Prediction

In this tutorial, we will demonstrate how to use a pretrained Cell2Sentence (C2S) model to perform cell type prediction on single-cell RNA sequencing datasets. Cell type prediction is a crucial step in single-cell analysis, allowing researchers to identify and classify different cell populations within a dataset. By leveraging the power of C2S models, we can make accurate predictions based on the information encoded in cell sentences.

In this tutorial, you will:
1. Load the PBMC 3K dataset from 10x (preprocessed in tutorial notebook 1)
2. Load a pretrained C2S model that is capable of making cell type predictions.
3. Use the model to predict cell types based on the cell sentences derived from the dataset.

We will begin by importing the necessary libraries. These include Python's built-in libraries, third-party libraries for handling numerical computations, progress tracking, and specific libraries for single-cell RNA sequencing data and C2S operations.

In [ ]:
# Python built-in libraries
import os
import pickle
import random
from collections import Counter

# Third-party libraries
import numpy as np
from tqdm import tqdm

# Single-cell libraries
import anndata
import scanpy as sc

# Cell2Sentence imports
import cell2sentence as cs
from cell2sentence.tasks import predict_cell_types_of_data

In [ ]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
import torch
torch.cuda.empty_cache()

# Load Data

Next, we will load the preprocessed dataset from the tutorial 0. This dataset has already been filtered and normalized, so it it ready for transformation into cell sentences.

<font color='red'>Please make sure you have completed the preprocessing steps in Tutorial 0 before running the following code, if you are using your own dataset.</font>. Ensure that the file path is correctly set in <font color='gold'>DATA_PATH</font> to where your preprocessed data was saved from tutorial 0.

In [ ]:
DATA_PATH = "./data/pbmc3k_final.h5ad"

In [ ]:
adata = anndata.read_h5ad(DATA_PATH)
adata

In [ ]:
adata.obs.head()

In [ ]:
#adata.var.head()

In [ ]:
sc.pl.umap(
    adata,
    color="cell_type",
    size=8,
    title="PBMC 3K UMAP",
)

# Cell2Sentence Conversion

In this section, we will transform our AnnData object containing our single-cell dataset into a Cell2Sentence (C2S) dataset by calling the functions of the CSData class in the C2S code base. Full documentation for the functions of the CSData class can be found in the documentation page of C2S.

In [ ]:
adata_obs_cols_to_keep = ["cell_type","organism"]

In [ ]:
# Create CSData object
arrow_ds, vocabulary = cs.CSData.adata_to_arrow(
    adata=adata, 
    random_state=SEED, 
    sentence_delimiter=' ',
    label_col_names=adata_obs_cols_to_keep
)

In [ ]:
arrow_ds

For this exercise we will consider the top 100 genes of the cell sentences

In [ ]:
k = 100  # replace with your desired number of genes

arrow_ds = arrow_ds.map(lambda x: {"cell_sentence": " ".join(x["cell_sentence"].split()[:k])})

In [ ]:
sample_idx = 2000
len(arrow_ds[sample_idx]['cell_sentence'].split())

In [ ]:
c2s_save_dir = "./c2s_api_testing"  # C2S dataset will be saved into this directory
c2s_save_name = "PBMC_3K_tutorial3"  # This will be the name of our C2S dataset on disk

In [ ]:
cs_data = cs.CSData.csdata_from_arrow(
    arrow_dataset=arrow_ds, 
    vocabulary=vocabulary,
    save_dir=c2s_save_dir,
    save_name=c2s_save_name,
    dataset_backend="arrow"
)

In [ ]:
print(cs_data)

In [ ]:
len(cs_data.get_sentence_strings())

This time, we will leave off creating our CSData object until after we load our C2S model. This is because along with the model checkpoint, we saved the indices of train, val, and test set cells, which will allow us to select out test set cells for inference.

# Load C2S Model

Now, we will load a C2S model with which we will do cell type annotation. For this tutorial, this model will be the last checkpoint of the training session from <font color="red">tutorial notebook 4</font>, where we finetuned our cell type prediction model to do cell type prediction specifically on our immune tissue dataset. We will load the last checkpoint saved from training, and specify the same save_dir as we used before during training.
- <font color="red">Note:</font> If you are using your own data for this tutorial, make sure to switch out to the model checkpoint which you saved in tutorial notebook 3.
- If you want to annotate cell types without finetuning your own C2S model, then tutorial notebook 6 demonstrates how to load the C2S-Pythia-410M cell type prediction foundation model and use it to predict cell types without any finetuning.

We can define our CSModel object with our pretrained cell type prediction model as follows, specifying the same save_dir as we used in tutorial 3:

In [ ]:
# Define CSModel object
cell_type_prediction_model_path = "./c2s_api_testing/csmodel_tutorial_3/2025-04-03-14_05_10_finetune_cell_type_prediction/checkpoint-330"

#save_dir = "/home/sr2464/palmer_scratch/C2S_Files_Syed/c2s_api_testing/csmodel_tutorial_3"
save_dir = "./c2s_api_testing/csmodel_tutorial_3"

save_name = "cell_type_pred_pythia_410M_2"
csmodel = cs.CSModel(
    model_name_or_path=cell_type_prediction_model_path,
    save_dir=save_dir,
    save_name=save_name
)

We will also load the data split indices saved alongside the C2S model checkpoint, so that we know which cells were part of the training and validation set. We will do inference on unseen test set cells, which are 10% of the original data.

In [ ]:
base_path = "/".join(cell_type_prediction_model_path.split("/")[:-1])
print(cell_type_prediction_model_path)
print(base_path)

In [ ]:
with open(os.path.join(base_path, 'data_split_indices_dict.pkl'), 'rb') as f:
    data_split_indices_dict = pickle.load(f)
data_split_indices_dict.keys()

In [ ]:
print(len(data_split_indices_dict["train"]))
print(len(data_split_indices_dict["val"]))
print(len(data_split_indices_dict["test"]))

Select out test set cells from full arrow dataset

In [ ]:
arrow_ds

In [ ]:
test_ds = arrow_ds.select(data_split_indices_dict["test"])
test_ds

Now, we will create our CSData object using only the test set cells:

In [ ]:
#c2s_save_dir = "/home/sr2464/palmer_scratch/C2S_Files_Syed/c2s_api_testing"  # C2S dataset will be saved into this directory
c2s_save_dir = "./c2s_api_testing"  # C2S dataset will be saved into this directory

c2s_save_name = "PBMC_3K_tutorial4"  # This will be the name of our C2S dataset on disk

In [ ]:
csdata = cs.CSData.csdata_from_arrow(
    arrow_dataset=test_ds, 
    vocabulary=vocabulary,
    save_dir=c2s_save_dir,
    save_name=c2s_save_name,
    dataset_backend="arrow"
)

In [ ]:
print(csdata)

# Predict cell types

Now that we have loaded our finetuned cell type prediction model and have our test set, we will do cell type prediction inference using our C2S model. We can use the function predict_cell_types_of_data() from the tasks.py, which will take a CSModel() object and apply it to do cell type prediction on a CSData() object.

In [ ]:
predicted_cell_types = predict_cell_types_of_data(
    csdata=csdata,
    csmodel=csmodel,
    n_genes=100
)

In [ ]:
len(predicted_cell_types)

In [ ]:
predicted_cell_types[:3]

In [ ]:
test_ds

In [ ]:
total_correct = 0.0
for model_pred, gt_label in zip(predicted_cell_types, test_ds["cell_type"]):
    # C2S might predict a period at the end of the cell type, which we remove
    if model_pred[-1] == ".":
        model_pred = model_pred[:-1]
    
    if model_pred == gt_label:
        total_correct += 1

accuracy = total_correct / len(predicted_cell_types)

In [ ]:
print("Accuracy:", accuracy)

In [ ]:
for idx in range(0, 100, 10):
    print("Model pred: {}, GT label: {}".format(predicted_cell_types[idx], test_ds[idx]["cell_type"]))

We can see that our model achieves high accuracy, correctly predicting the cell type of unseen cells from the immune tissue data 83% of the time! The model learned to predict cell type annotations in natural language effectively from a short finetuning period on the new data.


[Go to Notebook 6 →](./6_Synthetic_Cell_Generation.ipynb) for some, well, advanced use cases like the generation of synthetic cells.